## Введение в анализ данных, ИАД-2

## НИУ ВШЭ, 2018-19 учебный год

### Домашнее задание №3

Задание выполнил(а): Цю-жен-цин Дмитрий

### Общая информация

__Дата выдачи:__ 06.05.2019

__Дедлайн:__ 23:59 23.05.2019

### Оценивание и штрафы

Максимальное число баллов, которое можно получить за обязательную часть — 10, а максимальное число дополнительных баллов, которые пойдут в бонус — 0.5.

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-2: zCH4F32

In [268]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer, load_boston
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestClassifier, GradientBoostingRegressor
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold

# Random Forest своими руками (4)

Реализуйте алгоритм случайного леса. В соответствии с построением алгоритма, ваша реализация должна сочетать в себе метод бэггинга и метод случайных подмножеств. Также она должна работать для двух типов задач: как для задачи регрессии, так и для задачи классификации.

*Примечание. Вы можете использовать `DecisionTree` из `sklearn`.*

In [342]:
class RandomForest:
    def __init__(self, n_estimators, max_depth, max_features='sqrt', task_type='regression'):
        """
        PARAMETERS:
        n_estimators - number of trees in the ensemble
        max_depth - maximum depth of a tree
        max_features - maximum features for each split in DecisionTree from sklearn
        task_type - 'regression' or 'classification'
        """
        n_samples, self.n_features_ = X.shape
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        if isinstance(max_features, str):
            if max_features == "sqrt":
                self.max_features = max(1, int(np.sqrt(self.n_features_)))
            elif self.max_features == "log2":
                self.max_features = max(1, int(np.log2(self.n_features_)))
        elif isinstance(max_features, int):
            self.max_features = max_features
        else:  # float
            if max_features > 0.0:
                self.max_features = max(1, int(max_features * self.n_features_))
            else:
                self.max_features = 0

        
        self.task_type = task_type
        self.trees = []
        
        
    def fit(self, X_train, y_train):
        """
        INPUT:
        X_train - np.array of shape (k, d)
        y_train - np.array of shape (k,)
        """
        
        for i in range(self.n_estimators):
            idxs = np.random.permutation(X_train.shape[0])
            if self.task_type == 'regression':
                tree = DecisionTreeRegressor(max_features = self.max_features, max_depth = self.max_depth) 
                tree.fit(X_train[idxs], y_train[idxs])
                self.trees.append(tree)
            elif self.task_type == 'classification':
                tree = DecisionTreeClassifier(max_features = self.max_features, max_depth = self.max_depth)
                tree.fit(X_train[idxs], y_train[idxs])
                self.trees.append(tree)
            
          

        return self
        
    def predict(self, X_test):
        """
        INPUT:
        X_test - np.array of shape (m, d)
        
        OUTPUT:
        y_pred - np.array of shape (m,)
        """
        if self.task_type == 'regression':
            return np.mean([t.predict(X_test) for t in self.trees], axis = 0)
        elif self.task_type == 'classification':
            return (stats.mode([t.predict(X_test) for t in self.trees], axis = 0)[0])[0]
            

       

Проверьте вашу реализацию на задаче регрессии. Подберите оптимальные гиперпараметры, чтобы получить MSE < 10.25 **(не меняйте параметры сида)**.

In [375]:
boston = load_boston()
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [376]:
np.random.seed(19052019)

random_forest = RandomForest(n_estimators = 40, max_depth = 10, max_features = 'sqrt', task_type= 'regression')
random_forest.fit(X_train, y_train)
predicted_value = random_forest.predict(X_test)
mean_squared_error(y_test, predicted_value)


9.642785025285113

Проверьте вашу реализацию на задаче классификации. Подберите оптимальные гиперпараметры, чтобы получить accuracy > 0.83 **(не меняйте параметры сида)**.

In [377]:
breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [378]:
np.random.seed(19052019)
random_forest = RandomForest(20, 5, 'sqrt', 'classification')
random_forest.fit(X_train, y_train)
predicted_values = random_forest.predict(X_test)
accuracy_score(y_test, predicted_values)

0.951048951048951

# Градиентный бустинг своими руками (4)

Реализуйте алгоритм градиентного бустинга для регрессии. Напомним основные формулы.

Обозначим текущую композицию на $N-1$ шаге за $a_{N - 1}(x_i)$. Следующий базовый алгоритм $b_N(x_i)$ обучается на ответах $-\frac{\partial L(y_i, z)}{\partial z}\Bigl|_{z = a_{N - 1}(x_i)}$, где $L(y_i, z)$ — значение функции потерь на объекте при правильном ответе $y_i$ и предсказании $z$. Композиция на следующем шаге получается следующим образом:

$$
a_N(x_i) = a_{N-1}(x_i) + \nu\gamma_Nb_N(x_i)
$$

Здесь $\nu$ — гиперпараметр learning rate, $\gamma_N$ — оптимальный вес, настраиваемый на каждом шаге алгоритма, который можно найти по следующей формуле (обратите внимание на отсутствие $\nu$):

$$
\gamma_N = \mathrm{arg}\min_\gamma \frac{1}{\ell}\sum\limits_{i=1}^{\ell}L\left(y_i, a_{N - 1}(x_i) + \gamma b_N(x_i)\right)
$$

Можете принять $\gamma_N = 1$ для каждого $N$. Однако вы можете получить дополнительные 0.5 баллов за реализацию нахождения оптимального $\gamma_N$ на каждом шаге.

В качестве функции потерь возьмите MSE.

*Примечание. Вы можете использовать `DecisionTree` из `sklearn` и методы оптимизации из различных библиотек.*

In [379]:
class GradientBoosting:
    def __init__(self, n_estimators, max_depth, learning_rate=0.1):
        """
        PARAMETERS:
        n_estimators - number of trees in the ensemble
        max_depth - maximum depth of a tree
        learning_rate - coefficient by which new algorithm result is multiplied
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.trees = []
        self.mean_train = None
        for _ in range(n_estimators):
            tree = DecisionTreeRegressor(max_depth = self.max_depth, )
            self.trees.append(tree)
        
      
    def fit(self, X_train, y_train):
        self.mean_train = np.mean(y_train)
        y_pred = np.full(np.shape(y_train), np.mean(y_train, axis=0))
        for i in range(self.n_estimators):
            pseudo_residuals = -(y_train - y_pred)
            self.trees[i].fit(X_train, pseudo_residuals)
            new_base_prediction = self.trees[i].predict(X_train)
            y_opt = 1
            y_opt = np.sum(y_pred * new_base_prediction - new_base_prediction * y_train)/np.sum(np.power(new_base_prediction,2))
            y_pred = y_pred  - self.learning_rate * new_base_prediction * y_opt
                    
    def predict(self, X):
        y_pred = np.full(X.shape[0], self.mean_train)
        for tree in self.trees:
            update = tree.predict(X)
            update = np.multiply(self.learning_rate, update)
            y_pred = -update if not y_pred.any() else y_pred - update
        return y_pred
    
    

Проверьте вашу реализацию. Подберите оптимальные гиперпараметры, чтобы получить MSE < 10 как в обычном случае, так и при нахождении оптимального шага **(не меняйте параметры сида)**.

In [380]:
boston = load_boston()

X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [381]:
np.random.seed(19052019)
gb = GradientBoosting(n_estimators = 20, max_depth =6, learning_rate =0.30000000000000004)
gb.fit(X_train, y_train)
l = gb.predict(X_test)
mean_squared_error(y_test, l)

9.572959343867181

Поиск гиперпараметров (можно ли сделать это через GridSearch или RandomSearch?)

In [277]:
# np.random.seed(19052019)
# min_value = 20
# hyper = {0: 20}
# count = 0
# for i in range(0, 500, 10):
#     if i in range(10, 500, 40):
#         print(i)
#     for k in range(1, 20):
#         for m in np.linspace(0.1, 0.9, num = 5):
#             np.random.seed(19052019)
#             gb = GradientBoosting(i, k, learning_rate = m)
#             gb.fit(X_train, y_train)
#             my_predict = gb.predict(X_test)
#             mse = mean_squared_error(y_test, my_predict)
#             if  mse < min_value:
#                 min_value = mse
#                 hyper[i+k] = [i, k, m]

In [278]:
min_value

9.60715280944966

In [279]:
hyper

{0: 20, 10: [10, 6, 0.30000000000000004], 20: [20, 6, 0.30000000000000004]}

Для удобства проверяющего укажите, пожалуйста, реализовали ли вы нахождение оптимального шага?

- Да / Нет

In [280]:
 def fit(self, X_train, y_train):
        """
        INPUT:
        X_train - np.array of shape (k, d)
        y_train - np.array of shape (k,)
        """
        f = {}
        h = {}
        y_opt = 1
        f[0] = np.full(len(y_train), np.mean(y_train, axis = 0))
       
        for m in range(1, self.n_estimators): 
            residual = 2 * (y_train - f[m-1])
            tree = DecisionTreeRegressor(max_depth = self.max_depth)
            tree.fit(X_train, residual)
            self.trees.append(tree)
            h[m] = tree.predict(X_train)
            f[m] = f[m-1] + self.learning_rate* y_opt * h[m]
            
def predict(self, X_test):
        """
        INPUT:
        X_test - np.array of shape (m, d)
        
        OUTPUT:
        y_pred - np.array of shape (m,)
        """
        
        predict = np.zeros(X_test.shape[0]) 
        y_opt = 1
        for i in range(2, self.n_estimators):
            predict = predict + self.learning_rate * y_opt * self.trees[i-1].predict(X_test)
            
        return predict

# Крестики-нолики (2)

In [281]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

В этом задании вы поработаете с [данными](https://datahub.io/machine-learning/tic-tac-toe-endgame), состоящими из конечных позиций в игру крестики-нолики. Признаками являются клетки, их значениями — то, что изображено в данной клетке на момент окончания игры (крестик, нолик или пусто). Стоит задача бинарной классификации — определить, выиграли крестики (класс `True`) или нолики (класс `False`). Напомним, что одна из сторон выигрывает в том случае, если либо одна из диагоналей, либо одна из горизонталей, либо одна из вертикалей заполнена соответствующими символами.

In [282]:
data = pd.read_csv('tic-tac-toe.csv')
X = data.drop('class', axis=1)
y = data['class'].astype(int)
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [283]:
X = X.applymap(lambda y: 1 if y == 'x' else -1 if y == 'o' else 0)
X.head(15)

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1,1,1,1,-1,-1,1,-1,-1
1,1,1,1,1,-1,-1,-1,1,-1
2,1,1,1,1,-1,-1,-1,-1,1
3,1,1,1,1,-1,-1,-1,0,0
4,1,1,1,1,-1,-1,0,-1,0
5,1,1,1,1,-1,-1,0,0,-1
6,1,1,1,1,-1,0,-1,-1,0
7,1,1,1,1,-1,0,-1,0,-1
8,1,1,1,1,-1,0,0,-1,-1
9,1,1,1,1,0,-1,-1,-1,0


Применим метод случайного леса к данным и посмотрим на среднее значение accuracy на кросс-валидации.

In [284]:
rf = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=19052019)
score = cross_val_score(rf, X, y, cv=5, scoring='accuracy').mean()
print(score)

0.8917722942607311


Задача: улучшите данное значение до >0.945 с помощью Feature Engineering.

Добавим признаки, определяющие победу в классических крестиках-ноликах: по горизонталям, вертикалям и диагоналям.

In [285]:
X['1Hor'] = X['TL'] + X['TM'] + X['TR']
X['2Hor'] = X['ML'] + X['MM'] + X['MR']
X['3Hor'] = X['BL'] + X['BM'] + X['BR']
X['1Col'] = X['TL'] + X['ML'] + X['BL']
X['2Col'] = X['TM'] + X['MM'] + X['BM']
X['3Col'] = X['TR'] + X['MR'] + X['BR']
X['1Diag'] = X['TL'] + X['MM'] + X['BR']
X['2Diag'] = X['TR'] + X['MM'] + X['BL']

In [286]:
rf = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=19052019)
score = cross_val_score(rf, X, y, cv=5, scoring='accuracy').mean()
print(score)

0.9469766432466159
